In [17]:
%matplotlib

import numpy as np
import scipy as sci
from scipy.stats import norm
import matplotlib.pyplot as plt
import pandas as pd

'''
Markov chain that fits mean and standard deviation for a gaussian (normally distributed) sample
'''
np.random.seed(42)

# Produce synthetic data
mu = 3  # Mean
sd = 0.2  # SD
n = 1000  # Number of data points

data = np.random.normal(mu, sd, n)

real_mean = np.mean(data)
real_sd = np.std(data)
print (real_mean)
print (real_sd)


Using matplotlib backend: Qt5Agg
3.0038664111644646
0.19574524154947084


In [4]:
'''
Set up MCMC (Markov Chain Monte Carlo)
'''

# Starting guess [mu, sd]
mu_guess = np.random.random()
sd_guess = np.random.random()

init_param = np.array([0.80766639, 0.61260868])

# Proposal widths
w = [0.2, 0.2]


# Prior functions (prior beliefs in mu, sigma):
# That mu and sigma are between -10 and 10

# mu_prior_fun = sci.uniform(-10, 10) 
# uniform.rvs(size=n, loc = start, scale=width)
def mu_prior_fun(p):
    min_mu = -10
    max_mu = 10
    if (p > min_mu) and (p < max_mu):
        truth = 1
    else:
        truth = 0

    return truth / (max_mu - min_mu)


# sd_prior_fun =
def sd_prior_fun(p):
    min_sd = 0.01
    max_sd = 10
    if (p > min_sd) and (p < max_sd):
        truth = 1
    else:
        truth = 0

    return truth / (max_sd - min_sd)


prior_funcs = [mu_prior_fun, sd_prior_fun]

# Number of iterations
n_iterates = 500

param = init_param

# Define function to calculate log likelihood
def gaussian_ll(data, params):  # params = [mu, sd]
    mu = params[0]
    sd = params[1]

    ll = 0
    for i in data:
        ll = ll + norm.logpdf(i, loc=mu, scale=sd)

    return ll

# Calculate log likelihood (ll) of initial guess

ll = gaussian_ll(data, param)

# Establish data store for chain
# Where first column = ll
# And second column and onwards = model parameters (in this case 2 = mu, 3 = sigma)

chain = np.zeros(shape=(n_iterates, len(param) + 1))
chain[0, 0] = ll
chain[0, 1:] = param



In [5]:
# Run MCMC
prop_param = np.array([0, 0])

for i in range(n_iterates):
    if i % 10 == 0:
        print('Iteration of ' + str(i) + '/' + str(n_iterates))  # Print status every ten iterations

    for j in range(len(param)):  # Gibbs loop over number of parameters (i.e. j=1 is mu, j=2 is sd)
        # Propose a parameter value within prev. set widths
        prop_param = param.copy()

        if np.random.random() < 0.5:
            sign = 1
        else:
            sign = -1  # randomly plus or minus width (faster than random.choice())

        prop_param[j] = prop_param[j] - (sci.stats.uniform.rvs(loc=-w[j] / 2, scale=w[j], size=1))

        # Calculate log likelihood of proposal
        prop_ll = gaussian_ll(data, prop_param)

        # Accept or reject proposal
        prior_fun = prior_funcs[j]  # Grab the correct prior function (mu or sd)

        # Likelihood ratio
        r = np.exp(prop_ll - ll) * prior_fun(prop_param[j]) / prior_fun(param[j])
        # Is likelihood ratio less than or equal to one
        alpha = min(1, r)

        # Random number between 0 to 1
        # So will have weighted chance of possibly accepting depending on how likely the new parameter is
        test = np.random.uniform(0, 1)
        # Maybe accept
        if (test < alpha):
            ll = prop_ll
            param = prop_param.copy()
        # "Else" reject, though nothing to write

        # Store iterate
        chain[i, 0] = ll

        chain[i, 1:] = param



Iteration of 0/500
Iteration of 10/500
Iteration of 20/500
Iteration of 30/500
Iteration of 40/500
Iteration of 50/500
Iteration of 60/500
Iteration of 70/500
Iteration of 80/500
Iteration of 90/500
Iteration of 100/500
Iteration of 110/500
Iteration of 120/500
Iteration of 130/500
Iteration of 140/500
Iteration of 150/500
Iteration of 160/500
Iteration of 170/500
Iteration of 180/500
Iteration of 190/500
Iteration of 200/500
Iteration of 210/500
Iteration of 220/500
Iteration of 230/500
Iteration of 240/500
Iteration of 250/500
Iteration of 260/500
Iteration of 270/500
Iteration of 280/500
Iteration of 290/500
Iteration of 300/500
Iteration of 310/500
Iteration of 320/500
Iteration of 330/500
Iteration of 340/500
Iteration of 350/500
Iteration of 360/500
Iteration of 370/500
Iteration of 380/500
Iteration of 390/500
Iteration of 400/500
Iteration of 410/500
Iteration of 420/500
Iteration of 430/500
Iteration of 440/500
Iteration of 450/500
Iteration of 460/500
Iteration of 470/500
Ite

In [6]:
'''
Fitting a normal distribution 
(calculating likelihood with normal distribution) 
to a dataset generated from a normal distribution
'''
# Plot

chain = pd.DataFrame(chain, columns=['ll', 'mu', 'sd']) # Change np to panda array

n = np.arange(int(n_iterates)) # Creates array of iterates

chain.plot(kind='line', y = 'll')
plt.ylabel('Log Likelihood')
plt.xlabel('Iterate')

chain.plot(kind = 'line', y = 'mu')
plt.ylabel('Estimate for mu')
plt.xlabel('Iterate')

chain.plot(kind = 'line', y = 'sd')
plt.ylabel('Estimate for sd')
plt.xlabel('Iterate')

chain[['mu']].plot(kind = 'hist')

chain[['sd']].plot(kind = 'hist')

plt.show()


In [7]:
'''
Fitting a normal distribution 
(calculating likelihood with normal distribution) 
to a dataset generated from a normal distribution
'''

'''
Converges fairly closely to solved mu and sd value
SD diverges largely at first (moves sharply away from actual value before returning)
as curve widens to bring proposed mean closer to actual mean 
Once reaching close to actual mean, curve narrows and sd decreases and nears actual value
Also reflected in ll curve where ll not really increasing for a period
'''

'\nConverges fairly closely to solved mu and sd value\nSD diverges largely at first (moves sharply away from actual value before returning)\nas curve widens to bring proposed mean closer to actual mean \nOnce reaching close to actual mean, curve narrows and sd decreases and nears actual value\nAlso reflected in ll curve where ll not really increasing for a period\n'

In [8]:
'''
Fitting a poisson distribution to data generated from a normal distribution
'''

# Define function to calculate log likelihood
def poisson_ll(data, params):  # params = [mu]
    mu = params[0]

    ll = 0
    for i in data:
        ll = ll + sci.stats.poisson.logpmf(i, mu)

    return ll

'''
Set up MCMC (Markov Chain Monte Carlo)
'''

# Starting guess [mu]
mu_guess = np.random.random()

init_param = np.array([0.80766639])

# Proposal widths
w = [0.2, 0.2]


# Prior functions (prior beliefs in mu, sigma):
# That mu and sigma are between -10 and 10

# mu_prior_fun = sci.uniform(-10, 10)
# uniform.rvs(size=n, loc = start, scale=width)
def mu_prior_fun(p):
    min_mu = -10
    max_mu = 10
    if (p > min_mu) and (p < max_mu):
        truth = 1
    else:
        truth = 0

    return truth / (max_mu - min_mu)


prior_funcs = [mu_prior_fun]

# Number of iterations
n_iterates = 500
param = init_param

# Calculate log likelihood (ll) of initial guess

ll = poisson_ll(data, param)

# Establish data store for chain
# Where first column = ll
# And second column and onwards = model parameters (in this case 2 = mu, 3 = sigma)

chain = np.zeros(shape=(n_iterates, len(param) + 1))
chain[0, 0] = ll
chain[0, 1:] = param


In [9]:
'''
Fitting a poisson distribution to data generated from a normal distribution
'''

# Run MCMC
prop_param = np.array([0, 0])

for i in range(n_iterates):
    if i % 10 == 0:
        print('Iteration of ' + str(i) + '/' + str(n_iterates))  # Print status every ten iterations

    for j in range(len(param)):  # Gibbs loop over number of parameters (i.e. j=1 is mu, j=2 is sd)
        # Propose a parameter value within prev. set widths
        prop_param = param.copy()

        prop_param[j] = prop_param[j] - (sci.stats.uniform.rvs(loc=-w[j] / 2, scale=w[j], size=1))

        # Calculate log likelihood of proposal
        prop_ll = poisson_ll(data, prop_param)

        # Accept or reject proposal
        prior_fun = prior_funcs[j]  # Grab the correct prior function (mu or sd)

        # Likelihood ratio
        r = np.exp(prop_ll - ll) * prior_fun(prop_param[j]) / prior_fun(param[j])
        # Is likelihood ratio less than or equal to one
        alpha = min(1, r)

        # Random number between 0 to 1
        # So will have weighted chance of possibly accepting depending on how likely the new parameter is
        test = np.random.uniform(0, 1)
        # Maybe accept
        if (test < alpha):
            ll = prop_ll
            param = prop_param.copy()
        # "Else" reject, though nothing to write

        # Store iterate
        chain[i, 0] = ll
        chain[i, 1:] = param


Iteration of 0/500
Iteration of 10/500

/Users/caitlinng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars



Iteration of 20/500
Iteration of 30/500
Iteration of 40/500
Iteration of 50/500
Iteration of 60/500
Iteration of 70/500
Iteration of 80/500
Iteration of 90/500
Iteration of 100/500
Iteration of 110/500
Iteration of 120/500
Iteration of 130/500
Iteration of 140/500
Iteration of 150/500
Iteration of 160/500
Iteration of 170/500
Iteration of 180/500
Iteration of 190/500
Iteration of 200/500
Iteration of 210/500
Iteration of 220/500
Iteration of 230/500
Iteration of 240/500
Iteration of 250/500
Iteration of 260/500
Iteration of 270/500
Iteration of 280/500
Iteration of 290/500
Iteration of 300/500
Iteration of 310/500
Iteration of 320/500
Iteration of 330/500
Iteration of 340/500
Iteration of 350/500
Iteration of 360/500
Iteration of 370/500
Iteration of 380/500
Iteration of 390/500
Iteration of 400/500
Iteration of 410/500
Iteration of 420/500
Iteration of 430/500
Iteration of 440/500
Iteration of 450/500
Iteration of 460/500
Iteration of 470/500
Iteration of 480/500
Iteration of 490/500

In [10]:
'''
Fitting a poisson distribution to data generated from a normal distribution
'''

chain = pd.DataFrame(chain, columns=['ll', 'mu']) # Change np to panda array

n = np.arange(int(n_iterates)) # Creates array of iterates


# Plot

chain.plot(kind='line', y = 'll')
plt.ylabel('Log Likelihood')
plt.xlabel('Iterate')

chain.plot(kind = 'line', y = 'mu')
plt.ylabel('Estimate for mu')
plt.xlabel('Iterate')

chain[['mu']].plot(kind = 'hist')

plt.show()

print(np.mean(data))

3.0038664111644646


In [11]:
'''
Fitting a poisson distribution to data generated from a normal distribution
'''
'''
At n_iterates = 1000, doesn't reach convergence. ll still climbing up.
At n_iterates = 3000, breaks -- keeps going up far above computed mean.
Which makes sense since increasing the mean doesn't actually shift the whole curve right.
It just widens out the curve, which still starts at zero (google image).
'''

"\nAt n_iterates = 1000, doesn't reach convergence. ll still climbing up.\nAt n_iterates = 3000, breaks -- keeps going up far above computed mean.\nWhich makes sense since increasing the mean doesn't actually shift the whole curve right.\nIt just widens out the curve, which still starts at zero (google image).\n"

In [12]:
'''
Fitting a normal distribution to 
data generated from a poisson distribution
'''

# Produce synthetic data
mu = 3  # Mean
n = 1000  # Number of data points

data = np.random.poisson(mu, n)

real_mean = np.mean(data)
print (real_mean)

'''
Set up MCMC (Markov Chain Monte Carlo)
'''

# Starting guess [mu]
mu_guess = np.random.random()

init_param = np.array([0.80766639])

# Proposal widths
w = [0.2, 0.2]


# Prior functions (prior beliefs in mu, sigma):
# That mu and sigma are between -10 and 10

# mu_prior_fun = sci.uniform(-10, 10)
# uniform.rvs(size=n, loc = start, scale=width)
def mu_prior_fun(p):
    min_mu = -10
    max_mu = 10
    if (p > min_mu) and (p < max_mu):
        truth = 1
    else:
        truth = 0

    return truth / (max_mu - min_mu)


prior_funcs = [mu_prior_fun]

# Number of iterations
n_iterates = 500


# Define function to calculate log likelihood
def gaussian_ll(data, params):  # params = [mu]
    mu = params[0]

    ll = 0
    for i in data:
        ll = ll + norm.logpdf(i, loc=mu)

    return ll


# Calculate log likelihood (ll) of initial guess
param = init_param
ll = gaussian_ll(data, param)

# Establish data store for chain
# Where first column = ll
# And second column and onwards = model parameters (in this case 2 = mu, 3 = sigma)

chain = np.zeros(shape=(n_iterates, len(param) + 1))
chain[0, 0] = ll
chain[0, 1:] = param

# Run MCMC
prop_param = np.array([0, 0])

for i in range(n_iterates):
    if i % 10 == 0:
        print('Iteration of ' + str(i) + '/' + str(n_iterates))  # Print status every ten iterations

    for j in range(len(param)):  # Gibbs loop over number of parameters (i.e. j=1 is mu, j=2 is sd)
        # Propose a parameter value within prev. set widths
        prop_param = param.copy()

        prop_param[j] = prop_param[j] - (sci.stats.uniform.rvs(loc=-w[j] / 2, scale=w[j], size=1))

        # Calculate log likelihood of proposal
        prop_ll = gaussian_ll(data, prop_param)

        # Accept or reject proposal
        prior_fun = prior_funcs[j]  # Grab the correct prior function (mu or sd)

        # Likelihood ratio
        r = np.exp(prop_ll - ll) * prior_fun(prop_param[j]) / prior_fun(param[j])
        # Is likelihood ratio less than or equal to one
        alpha = min(1, r)

        # Random number between 0 to 1
        # So will have weighted chance of possibly accepting depending on how likely the new parameter is
        test = np.random.uniform(0, 1)
        # Maybe accept
        if (test < alpha):
            ll = prop_ll
            param = prop_param.copy()
        # "Else" reject, though nothing to write

        # Store iterate
        chain[i, 0] = ll
        chain[i, 1:] = param





2.948
Iteration of 0/500
Iteration of 10/500
Iteration of 20/500
Iteration of 30/500
Iteration of 40/500
Iteration of 50/500
Iteration of 60/500
Iteration of 70/500
Iteration of 80/500
Iteration of 90/500
Iteration of 100/500
Iteration of 110/500
Iteration of 120/500
Iteration of 130/500
Iteration of 140/500
Iteration of 150/500
Iteration of 160/500
Iteration of 170/500
Iteration of 180/500
Iteration of 190/500
Iteration of 200/500
Iteration of 210/500
Iteration of 220/500
Iteration of 230/500
Iteration of 240/500
Iteration of 250/500
Iteration of 260/500
Iteration of 270/500
Iteration of 280/500
Iteration of 290/500
Iteration of 300/500
Iteration of 310/500
Iteration of 320/500
Iteration of 330/500
Iteration of 340/500
Iteration of 350/500
Iteration of 360/500
Iteration of 370/500
Iteration of 380/500
Iteration of 390/500
Iteration of 400/500
Iteration of 410/500
Iteration of 420/500
Iteration of 430/500
Iteration of 440/500
Iteration of 450/500
Iteration of 460/500
Iteration of 470/5

In [13]:
chain = pd.DataFrame(chain, columns=['ll', 'mu']) # Change np to panda array

n = np.arange(int(n_iterates)) # Creates array of iterates


# Plot

chain.plot(kind='line', y = 'll')
plt.ylabel('Log Likelihood')
plt.xlabel('Iterate')

chain.plot(kind = 'line', y = 'mu')
plt.ylabel('Estimate for mu')
plt.xlabel('Iterate')

chain[['mu']].plot(kind = 'hist')

plt.show()


In [14]:
'''
Fitting a normal distribution to data generated from a poisson distribution
'''

'''
Converges less smoothly to the actual value (more jagged plateau). No issues with needing to fit 2nd sd parameter though. So ll curve logarithmic, steady increase in likelihood and then plateauing off. ll also generally lower than fitting normal to normal though (about -2500 vs 0)
'''

'\nConverges less smoothly to the actual value (more jagged plateau). No issues with needing to fit 2nd sd parameter though. So ll curve logarithmic, steady increase in likelihood and then plateauing off. ll also generally lower than fitting normal to normal though (about -2500 vs 0)\n'

In [15]:
'''
Fitting a poisson distribution to data generated from a poisson distribution
'''
# Produce synthetic data
mu = 3  # Mean
n = 1000  # Number of data points

data = np.random.poisson(mu, n)

real_mean = np.mean(data)
print (real_mean)


# Define function to calculate log likelihood
def poisson_ll(data, params):  # params = [mu]
    mu = params[0]

    ll = 0
    for i in data:
        ll = ll + sci.stats.poisson.rvs(mu)

    return ll

'''
Set up MCMC (Markov Chain Monte Carlo)
'''

# Starting guess [mu]
mu_guess = np.random.random()

init_param = np.array([0.80766639])

# Proposal widths
w = [0.2, 0.2]


# Prior functions (prior beliefs in mu, sigma):
# That mu and sigma are between -10 and 10

# mu_prior_fun = sci.uniform(-10, 10)
# uniform.rvs(size=n, loc = start, scale=width)
def mu_prior_fun(p):
    min_mu = -10
    max_mu = 10
    if (p > min_mu) and (p < max_mu):
        truth = 1
    else:
        truth = 0

    return truth / (max_mu - min_mu)


prior_funcs = [mu_prior_fun]

# Number of iterations
n_iterates = 500


# Calculate log likelihood (ll) of initial guess
param = init_param
ll = poisson_ll(data, param)

# Establish data store for chain
# Where first column = ll
# And second column and onwards = model parameters (in this case 2 = mu, 3 = sigma)

chain = np.zeros(shape=(n_iterates, len(param) + 1))
chain[0, 0] = ll
chain[0, 1:] = param

# Run MCMC
prop_param = np.array([0, 0])

for i in range(n_iterates):
    if i % 10 == 0:
        print('Iteration of ' + str(i) + '/' + str(n_iterates))  # Print status every ten iterations

    for j in range(len(param)):  # Gibbs loop over number of parameters (i.e. j=1 is mu, j=2 is sd)
        # Propose a parameter value within prev. set widths
        prop_param = param.copy()

        prop_param[j] = prop_param[j] - (sci.stats.uniform.rvs(loc=-w[j] / 2, scale=w[j], size=1))

        # Calculate log likelihood of proposal
        prop_ll = poisson_ll(data, prop_param)

        # Accept or reject proposal
        prior_fun = prior_funcs[j]  # Grab the correct prior function (mu or sd)

        # Likelihood ratio
        r = np.exp(prop_ll - ll) * prior_fun(prop_param[j]) / prior_fun(param[j])
        # Is likelihood ratio less than or equal to one
        alpha = min(1, r)

        # Random number between 0 to 1
        # So will have weighted chance of possibly accepting depending on how likely the new parameter is
        test = np.random.uniform(0, 1)
        # Maybe accept
        if (test < alpha):
            ll = prop_ll
            param = prop_param.copy()
        # "Else" reject, though nothing to write

        # Store iterate
        chain[i, 0] = ll
        chain[i, 1:] = param


2.981
Iteration of 0/500
Iteration of 10/500
Iteration of 20/500
Iteration of 30/500
Iteration of 40/500
Iteration of 50/500
Iteration of 60/500
Iteration of 70/500
Iteration of 80/500
Iteration of 90/500
Iteration of 100/500
Iteration of 110/500
Iteration of 120/500
Iteration of 130/500
Iteration of 140/500
Iteration of 150/500
Iteration of 160/500
Iteration of 170/500
Iteration of 180/500
Iteration of 190/500
Iteration of 200/500
Iteration of 210/500
Iteration of 220/500
Iteration of 230/500
Iteration of 240/500
Iteration of 250/500
Iteration of 260/500
Iteration of 270/500
Iteration of 280/500
Iteration of 290/500
Iteration of 300/500
Iteration of 310/500
Iteration of 320/500
Iteration of 330/500
Iteration of 340/500
Iteration of 350/500
Iteration of 360/500
Iteration of 370/500
Iteration of 380/500
Iteration of 390/500
Iteration of 400/500
Iteration of 410/500
Iteration of 420/500
Iteration of 430/500
Iteration of 440/500
Iteration of 450/500
Iteration of 460/500
Iteration of 470/5

In [16]:
chain = pd.DataFrame(chain, columns=['ll', 'mu']) # Change np to panda array

n = np.arange(int(n_iterates)) # Creates array of iterates


# Plot

chain.plot(kind='line', y = 'll')
plt.ylabel('Log Likelihood')
plt.xlabel('Iterate')

chain.plot(kind = 'line', y = 'mu')
plt.ylabel('Estimate for mu')
plt.xlabel('Iterate')

chain[['mu']].plot(kind = 'hist')

plt.show()